# L5: Agentic RAG & External Memory


## Preparation

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>
</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI models can vary with each execution due to their dynamic, probabilistic nature. Your results may differ from those shown in the video.</p>

Letta agents persist information over time and restarts by saving data to a database. These lessons do not require past information. To enable a clean restart, the database is cleared before starting the lesson.

In [1]:
!rm  -f ~/.letta/sqlite.db

## Section 0: Setup a client 

In [9]:
import sys

sys.path[:0] = ["../scripts/"]

In [ ]:
from helper import nb_print, load_env
from dotenv import load_dotenv

_ = load_dotenv()

In [10]:
from helper import print_messages as nb_print

In [1]:
from letta import create_client 

client = create_client()

In [2]:
from letta.schemas.llm_config import LLMConfig
from letta.schemas.embedding_config import EmbeddingConfig

client.set_default_llm_config(LLMConfig(
    model="qwq:latest",
    model_endpoint_type="ollama",
    model_endpoint="http://localhost:11434",
    context_window=8192,
))
client.set_default_embedding_config(
    EmbeddingConfig.default_config(model_name="letta")
)

## Section 1: Loading data into archival memory 

In [3]:
source = client.create_source("employee_handbook")
source

Source(id='source-b91302d0-7861-49fb-b568-202d04914fd2', name='employee_handbook', description=None, embedding_config=EmbeddingConfig(embedding_endpoint_type='hugging-face', embedding_endpoint='https://embeddings.memgpt.ai', embedding_model='BAAI/bge-large-en-v1.5', embedding_dim=1024, embedding_chunk_size=300, azure_endpoint=None, azure_version=None, azure_deployment=None), organization_id='org-00000000-0000-4000-8000-000000000000', metadata_=None, created_by_id='user-00000000-0000-4000-8000-000000000000', last_updated_by_id='user-00000000-0000-4000-8000-000000000000', created_at=datetime.datetime(2024, 12, 19, 13, 52, 26), updated_at=datetime.datetime(2024, 12, 19, 13, 52, 26))

In [4]:
client.load_file_to_source(
    filename="../scripts/handbook.pdf", 
    source_id=source.id
)

Job(created_by_id='user-00000000-0000-4000-8000-000000000000', last_updated_by_id='user-00000000-0000-4000-8000-000000000000', created_at=datetime.datetime(2024, 12, 19, 13, 52, 27), updated_at=datetime.datetime(2024, 12, 19, 13, 52, 27), status=<JobStatus.created: 'created'>, completed_at=None, metadata_={'type': 'embedding', 'filename': '../scripts/handbook.pdf', 'source_id': 'source-b91302d0-7861-49fb-b568-202d04914fd2'}, id='job-423e4a06-7fc3-457e-a0a8-ba6a74c71a6d', user_id='user-00000000-0000-4000-8000-000000000000')

In [5]:
agent_state = client.create_agent()

In [6]:
client.attach_source_to_agent(
    agent_id=agent_state.id, 
    source_id=source.id
)

In [7]:
client.list_attached_sources(agent_state.id)

[Source(id='source-b91302d0-7861-49fb-b568-202d04914fd2', name='employee_handbook', description=None, embedding_config=EmbeddingConfig(embedding_endpoint_type='hugging-face', embedding_endpoint='https://embeddings.memgpt.ai', embedding_model='BAAI/bge-large-en-v1.5', embedding_dim=1024, embedding_chunk_size=300, azure_endpoint=None, azure_version=None, azure_deployment=None), organization_id='org-00000000-0000-4000-8000-000000000000', metadata_=None, created_by_id='user-00000000-0000-4000-8000-000000000000', last_updated_by_id='user-00000000-0000-4000-8000-000000000000', created_at=datetime.datetime(2024, 12, 19, 13, 52, 26), updated_at=datetime.datetime(2024, 12, 19, 13, 52, 26))]

In [ ]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "Search archival for our company's vacation policies", 
    role = "user"
) 

In [11]:
nb_print(response.messages)

INTERNAL_MONOLOGUE
I need to find our company's vacation policies in the archival memory. I'll search for 'vacation policies' to see if there's any relevant information stored.
----------------------------------------
FUNCTION_CALL
[archival_memory_search]
{'query': 'vacation policies', 'request_heartbeat': True}
----------------------------------------
INTERNAL_MONOLOGUE
Hmm, the archival memory search didn't turn up any results for 'vacation policies'. Maybe I need to try a different approach or check if the information is stored elsewhere.
----------------------------------------
FUNCTION_CALL
[send_message]
{'message': "I'm sorry, but I couldn't find any vacation policy information in my archival memory. Perhaps you can provide more details or check with HR for the latest policies."}
----------------------------------------


## Section 2: Connecting data via tools

In [12]:
def query_birthday_db(self, name: str): 
    """
    This tool queries an external database to 
    lookup the birthday of someone given their name.

    Args: 
        name (str): The name to look up 

    Returns: 
        birthday (str): The birthday in mm-dd-yyyy format
    
    """
    my_fake_data = {
        "bob": "03-06-1997", 
        "sarah": "03-06-1997"
    } 
    name = name.lower() 
    if name not in my_fake_data: 
        return None
    else: 
        return my_fake_data[name]

In [13]:
birthday_tool = client.create_tool(query_birthday_db)

In [14]:
birthday_tool

Tool(id='tool-50728de9-ffec-48ae-a410-5fc0351be387', description='This tool queries an external database to ', source_type='python', module=None, organization_id='org-00000000-0000-4000-8000-000000000000', name='query_birthday_db', tags=[], source_code='def query_birthday_db(self, name: str): \n    """\n    This tool queries an external database to \n    lookup the birthday of someone given their name.\n\n    Args: \n        name (str): The name to look up \n\n    Returns: \n        birthday (str): The birthday in mm-dd-yyyy format\n\n    """\n    my_fake_data = {\n        "bob": "03-06-1997", \n        "sarah": "03-06-1997"\n    } \n    name = name.lower() \n    if name not in my_fake_data: \n        return None\n    else: \n        return my_fake_data[name]\n', json_schema={'name': 'query_birthday_db', 'description': 'This tool queries an external database to ', 'parameters': {'type': 'object', 'properties': {'name': {'type': 'string', 'description': 'The name to look up '}, 'request

In [17]:
from letta.schemas.memory import ChatMemory

agent_state = client.create_agent(
    name="birthday_agent", 
    tool_ids=[birthday_tool.id,],
    memory=ChatMemory(
        human="My name is Sarah", 
        persona="You are a agent with access to a birthday_db " \
        + "that you use to lookup information about users' birthdays."
    )
)

In [18]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "When is my birthday?", 
    role = "user"
) 
nb_print(response.messages)

INTERNAL_MONOLOGUE
User asked when their birthday is. I need to check the database for Sarah's birthday.
----------------------------------------
FUNCTION_CALL
[query_birthday_db]
{'name': 'Sarah', 'request_heartbeat': True}
----------------------------------------
INTERNAL_MONOLOGUE
I need to handle the error from the database query. Maybe the database isn't set up correctly.
----------------------------------------
FUNCTION_CALL
[send_message]
{'message': "Sorry, I'm having trouble accessing that information right now. Let's try something else."}
----------------------------------------


### Loading tools from Langchain

In [ ]:
import getpass
import os

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key:\n")

In [ ]:
from langchain_community.tools import TavilySearchResults
from letta.schemas.tool import Tool

search = TavilySearchResults()

In [ ]:
search.run("What's Obama's first name?") 

In [ ]:
# convert the tool to Letta Tool 
search_tool = Tool.from_langchain(TavilySearchResults())

In [ ]:
# persist the tool 
client.add_tool(search_tool)

In [ ]:
research_agent_persona = f"""
You have access to a web via a {search_tool.name} tool. 
Use this tool to respond to users' questions, by summarizing the
{search_tool.name} 
and also providing the `url` that the information was from as a 
reference. 

<Example> 
User: 'What is Obama's first name?' 
Assistant: 'Obama's first name is Barack.

Sources:
[1] https://www.britannica.com/biography/Barack-Obama
[2] https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States'
</Example>
Your MUST provide URLs that you used to generate the answer, or you will be terminated. 

"""

In [ ]:
agent_state = client.create_agent(
    name="research_agent", 
    tool_ids==[search_tool.id], 
    memory=ChatMemory(
        human="My name is Sarah", 
        persona=research_agent_persona
    )
)

In [ ]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "Who founded OpenAI? ", 
    role = "user"
) 
nb_print(response.messages)

In [ ]:
from letta.schemas.llm_config import LLMConfig

agent_state = client.create_agent(
    name="gpt4_search_agent", 
    tools=[search_tool.name], 
    memory=ChatMemory(
        human="My name is Sarah", 
        persona=research_agent_persona
    ),
    #llm_config=LLMConfig.default_config('gpt-4')  # uncomment if you are not getting desired results. Note that this is much more expensive than gpt-4o-mini
    llm_config=LLMConfig.default_config('gpt-4o-mini')
)

In [ ]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "Who founded OpenAI? ", 
    role = "user"
) 
nb_print(response.messages)